In [1]:
# bridge related stats

In [2]:
import pandas as pd

In [10]:
df = pd.read_csv(r'../data/bridge/bridge_results.csv')

In [14]:
events = pd.read_csv(r'../data/bridge/bridge_status.csv')

In [158]:
events.iloc[0:10]

,bridge,time,event,date,text,newtime,timeobj
0,Fremont,5:09:25 PM,open,2016-01-31,The Fremont Bridge has reopened to traffic - ...,5:09:25 PM,1900-01-01 17:09:25
1,Fremont,4:59:02 PM,closed,2016-01-31,The Fremont Bridge has closed to traffic - 4...,4:59:02 PM,1900-01-01 16:59:02
2,Montlake,4:23:13 PM,open,2016-01-31,The Montlake Bridge has reopened to traffic ...,4:23:13 PM,1900-01-01 16:23:13
3,Montlake,4:17:50 PM,closed,2016-01-31,The Montlake Bridge has closed to traffic - ...,4:17:50 PM,1900-01-01 16:17:50
4,University,4:15:49 PM,open,2016-01-31,The University Bridge has reopened to traffi...,4:15:49 PM,1900-01-01 16:15:49
5,University,4:09:37 PM,closed,2016-01-31,The University Bridge has closed to traffic ...,4:09:37 PM,1900-01-01 16:09:37
6,Ballard,4:06:20 PM,open,2016-01-31,The Ballard Bridge has reopened to traffic - ...,4:06:20 PM,1900-01-01 16:06:20
7,Ballard,4:04:21 PM,closed,2016-01-31,The Ballard Bridge has closed to traffic - 4...,4:04:21 PM,1900-01-01 16:04:21
8,Fremont,3:55:26 PM,open,2016-01-30,The Fremont Bridge has reopened to traffic - ...,3:55:26 PM,1900-01-01 15:55:26
9,Fremont,3:44:55 PM,closed,2016-01-30,The Fremont Bridge has closed to traffic - 3...,3:44:55 PM,1900-01-01 15:44:55


# average closures 

In [99]:
closures = events[events['event'] == 'closed']
closures_group = closures.groupby(['date','bridge']).count()

In [100]:
closures_group['date'] = closures_group.index.get_level_values(0)
closures_group['bridge'] = closures_group.index.get_level_values(1)
# closures_group[closures_group['bridge'] == 'Ballard']

In [101]:
avg_closure_per_day = {}
for bridge in closures_group.groupby('bridge').count().index:
    avg_closure_per_day[bridge] = closures_group[closures_group['bridge'] == bridge].mean()['event']

In [113]:
df = pd.DataFrame(avg_closure_per_day, index=avg_closure_per_day.keys()).T
df = pd.DataFrame(df['Ballard'])
df.columns = ['avg_closures']
df['bridge'] = df.index
df.to_csv('../data/bridge/avg_closures.csv')

In [114]:
# Get more sophisticated later; add average closures by season, time of day, etc

# closures in current day

In [141]:
import datetime
import math
# load file created by bridge_stat.closures_per_day()
recent_df = pd.read_csv(r'../data/bridge/recent_tweets.csv')

In [142]:
recent_df['local_date'] = pd.to_datetime(recent_df['local_date'])

In [143]:
today = datetime.datetime.today()

In [144]:
tweets_today = recent_df[recent_df['local_date'].apply(lambda row:
                       (row.day == today.day) &
                       (row.month == today.month) &
                       (row.year == today.year))
   ]

In [145]:
# Get a count of tweets by bridge 
tweets_today
closures_today = tweets_today[tweets_today['event'] == 'closed']
closures_by_bridge = closures_today.groupby('bridge').count()
closures_by_bridge['bridge'] = closures_by_bridge.index
closures_by_bridge = closures_by_bridge[['bridge','event']]
closures_by_bridge.columns = ['bridge','closures_so_far']

In [152]:
# Compare to average to get anticipated remained on average
remaining_closures_df = pd.merge(closures_by_bridge, df, left_on='bridge', right_on='bridge')

# Round up for average closures
remaining_closures_df['avg_closures'] = remaining_closures_df['avg_closures'].apply(lambda row: math.ceil(row))
remaining_closures_df['closures_remaining'] = remaining_closures_df['avg_closures'] - remaining_closures_df['closures_so_far']
remaining_closures_df['%_closures_remaining'] = remaining_closures_df['closures_remaining']/remaining_closures_df['avg_closures']

In [155]:
# Leave the negatives in: indicates odds less likely 
remaining_closures_df.to_csv('../data/bridge/closure_estimate.csv')

# Average time per closing

In [194]:
from datetime import datetime, timedelta
df = pd.read_csv('../data/bridge/bridge_records.csv')

In [196]:
def set_local_time(obj, offset_hours=-7):

    #account for offset from UTC using timedelta                                
    local_timestamp = obj + timedelta(hours=offset_hours)

    return local_timestamp

In [202]:
# Load all the historical tweets and process to get columns from text, convert to local time
df['bridge'] = df['text'].map(lambda x: x.split('The ')[-1].split(' Bridge')[0])
df['created_at'] = pd.to_datetime(df['created_at'])
df['local_date'] = df['created_at'].apply(lambda x: set_local_time(x))
df['event'] = df['text'].apply(lambda x: 'closed' if ('closed' in x) else 'open')
df = df[['bridge','event', 'text','created_at', 'local_date']]

bridge_list = ['Ballard', 'Fremont', '1st Ave S', 'Montlake', 'Lower Spokane St', 'South Park', 'University']

# Select only rows with a bridge name in the bridge list
df = df[df['bridge'].isin(bridge_list)]

In [203]:
# Do some analysis

In [280]:
fremont = df[df['bridge'] == 'Fremont']

# Sorting by date, should have an open, followed by close event
fremont = fremont.sort('local_date')
fremont.reset_index(inplace=True, drop=True)

# start with a closure event, and end with an 'open'
if fremont['event'].iloc[0] == 'open':
    fremont = fremont.drop(0)

if fremont['event'].iloc[-1] == 'closed':
    print 'dropping last'
    fremont = fremont.drop(fremont.index[-1])
    
# Make sure events are closed, followed by open
# all odds indices should be closed, events should be open
# other logic checks too?
# odds = open?


/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,bridge,event,text,created_at,local_date
27,Fremont,closed,The Fremont Bridge has closed to traffic - 9...,2015-11-12 17:23:46,2015-11-12 10:23:46
28,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-12 17:26:18,2015-11-12 10:26:18
29,Fremont,closed,The Fremont Bridge has closed to traffic - 1...,2015-11-12 21:07:55,2015-11-12 14:07:55
30,Fremont,closed,The Fremont Bridge has closed to traffic - 1...,2015-11-13 18:03:50,2015-11-13 11:03:50
31,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-13 18:07:32,2015-11-13 11:07:32
32,Fremont,closed,The Fremont Bridge has closed to traffic - 1...,2015-11-14 19:36:46,2015-11-14 12:36:46
33,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-14 19:39:37,2015-11-14 12:39:37
34,Fremont,closed,The Fremont Bridge has closed to traffic - 1...,2015-11-15 20:18:00,2015-11-15 13:18:00
35,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-15 20:23:32,2015-11-15 13:23:32


In [269]:
fremont.index.values

array([   2,    4,    6, ..., 5978, 5980, 5982])

,bridge,event,text,created_at,local_date
2,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-09 19:56:32,2015-11-09 12:56:32
4,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-09 20:24:38,2015-11-09 13:24:38
6,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-09 22:28:30,2015-11-09 15:28:30
8,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-09 23:02:17,2015-11-09 16:02:17
10,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-10 02:25:51,2015-11-09 19:25:51
12,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-10 03:16:44,2015-11-09 20:16:44
14,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-10 17:52:24,2015-11-10 10:52:24
16,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-10 22:11:11,2015-11-10 15:11:11
18,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-11 18:18:27,2015-11-11 11:18:27
20,Fremont,open,The Fremont Bridge has reopened to traffic - ...,2015-11-11 19:35:28,2015-11-11 12:35:28
